# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не
import sys
!{sys.executable} -m pip install -q requests beautifulsoup4 lxml schedule pytest numpy



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.13/bin/python3 -m pip install --upgrade pip


In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
 def get_book_data(book_url: str) -> dict:
    """
    Извлекает полную информацию о книге с её страницы на сайте Books to Scrape.

    Функция выполняет HTTP-запрос по указанному адресу, парсит HTML-страницу
    и возвращает все данные о книге — от основного блока с названием, ценой
    и рейтингом до сведений из таблицы Product Information.

    Parameters
    ----------
    book_url : str
        Полный URL страницы книги.

    Returns
    -------
    dict
        Словарь с характеристиками книги:
        - title : название книги
        - price : цена в фунтах
        - rating : рейтинг (One, Two, Three, Four, Five)
        - availability : информация о наличии
        - description : краткое описание книги
        - upc, product_type, price_excl_tax, price_incl_tax, tax,
          availability_count, number_of_reviews : данные из таблицы
          Product Information.

    Examples
    --------
    >>> url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
    >>> data = get_book_data(url)
    >>> print(data["title"])
    A Light in the Attic
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    response = requests.get(book_url, timeout=10)
    response.encoding = "utf-8"  # исправляем возможную ошибку декодирования
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "lxml")

    title = soup.find("div", class_="product_main").h1.get_text(strip=True)
    price = soup.find("p", class_="price_color").get_text(strip=True)
    availability = soup.find(
        "p", class_="instock availability"
    ).get_text(strip=True)
    rating = soup.find("p", class_="star-rating")["class"][1]

    desc_tag = soup.find("div", id="product_description")
    description = (
        desc_tag.find_next_sibling("p").get_text(strip=True)
        if desc_tag
        else ""
    )

    info_table = soup.find("table", class_="table table-striped")
    info = {
        row.th.get_text(strip=True): row.td.get_text(strip=True)
        for row in info_table.find_all("tr")
    }

    return {
        "title": title,
        "price": price,
        "availability": availability,
        "rating": rating,
        "description": description,
        "upc": info.get("UPC", ""),
        "product_type": info.get("Product Type", ""),
        "price_excl_tax": info.get("Price (excl. tax)", ""),
        "price_incl_tax": info.get("Price (incl. tax)", ""),
        "tax": info.get("Tax", ""),
        "availability_count": info.get("Availability", ""),
        "number_of_reviews": info.get("Number of reviews", ""),
    }

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
book = get_book_data(book_url)
print("\n📘 Информация о книге")
print("=" * 60)
print(f"Название:              {book['title']}")
print(f"Цена:                  {book['price']}")
print(f"Рейтинг:               {book['rating']}")
print(f"Наличие:               {book['availability']}")
print(f"Описание:              {book['description'][:120]}...")
print("-" * 60)
print("Дополнительные сведения:")
print(f"  • UPC:                {book['upc']}")
print(f"  • Тип продукта:       {book['product_type']}")
print(f"  • Цена без налога:    {book['price_excl_tax']}")
print(f"  • Цена с налогом:     {book['price_incl_tax']}")
print(f"  • Налог:              {book['tax']}")
print(f"  • Остаток на складе:  {book['availability_count']}")
print(f"  • Отзывов:            {book['number_of_reviews']}")
print("=" * 60)


📘 Информация о книге
Название:              A Light in the Attic
Цена:                  £51.77
Рейтинг:               Three
Наличие:               In stock (22 available)
Описание:              It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel ...
------------------------------------------------------------
Дополнительные сведения:
  • UPC:                a897fe39b1053632
  • Тип продукта:       Books
  • Цена без налога:    £51.77
  • Цена с налогом:     £51.77
  • Налог:              £0.00
  • Остаток на складе:  In stock (22 available)
  • Отзывов:            0


## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(is_save: bool = True) -> list[dict]:
    """
    Проходит по всем страницам каталога Books to Scrape и собирает
    данные о каждой книге, используя функцию get_book_data().

    Функция может сохранять результаты в файл books_data.txt
    (если установлен флаг is_save=True).

    Parameters
    ----------
    is_save : bool, optional
        Если True — сохраняет данные в файл 'books_data.txt',
        иначе возвращает список без сохранения (по умолчанию True).

    Returns
    -------
    list[dict]
        Список словарей с информацией о книгах, включая поля:
        title, price, rating, availability, description, upc и др.

    Examples
    --------
    >>> books = scrape_books(is_save=False)
    >>> len(books)
    1000

    Notes
    -----
    Сайт содержит 50 страниц по 20 книг.
    Между запросами добавлена задержка для снижения нагрузки на сервер.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    base_url = "http://books.toscrape.com/catalogue/page-{}.html"
    all_books = []

    page = 1
    while True:
        # формируем ссылку на страницу каталога
        page_url = base_url.format(page)
        response = requests.get(page_url)
        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, "html.parser")
        book_links = [
            "http://books.toscrape.com/catalogue/" + a["href"].replace("../../", "")
            for a in soup.select("h3 > a")
        ]

        if not book_links:
            break

        for link in book_links:
            try:
                book_data = get_book_data(link)
                all_books.append(book_data)
                print(f"  ✓ {book_data['title']}")
                time.sleep(0.2)  # чтобы не спамить запросами
            except Exception as exc:
                print(f"⚠️ Ошибка при обработке {link}: {exc}")

        next_button = soup.select_one("li.next > a")
        if not next_button:
            break
        page += 1

    if is_save:
        with open("books_data.txt", "w", encoding="utf-8") as file:
            for book in all_books:
                file.write(str(book) + "\n")
        print(f"\n✅ Данные сохранены в файл books_data.txt ({len(all_books)} записей).")

    return all_books
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
print("🚀 Запуск парсинга всех страниц каталога...")
res = scrape_books(is_save=True) # Допишите ваши аргументы
print("\n📊 Результаты парсинга")
print(f"Тип данных: {type(res)}")
print(f"Количество собранных книг: {len(res)}")
print(f"Пример записи:\n{res[0] if res else 'Нет данных'}")


🚀 Запуск парсинга всех страниц каталога...
  ✓ A Light in the Attic
  ✓ Tipping the Velvet
  ✓ Soumission
  ✓ Sharp Objects
  ✓ Sapiens: A Brief History of Humankind
  ✓ The Requiem Red
  ✓ The Dirty Little Secrets of Getting Your Dream Job
  ✓ The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
  ✓ The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
  ✓ The Black Maria
  ✓ Starving Hearts (Triangular Trade Trilogy, #1)
  ✓ Shakespeare's Sonnets
  ✓ Set Me Free
  ✓ Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
  ✓ Rip it Up and Start Again
  ✓ Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
  ✓ Olio
  ✓ Mesaerion: The Best Science Fiction Stories 1800-1849
  ✓ Libertarianism for Beginners
  ✓ It's Only the Himalayas
  ✓ In Her Wake
  ✓ How Music Works
  ✓ Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide t

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [7]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def job() -> None:
    """
    Единичный запуск задачи парсинга каталога книг.

    Запускает функцию scrape_books() и сохраняет
    обновлённые данные в файл books_data.txt.

    Notes
    -----
    Используется в качестве задачи для планировщика schedule.
    """
    print("\n🕖 Запуск задачи сбора данных...")
    scrape_books(is_save=True)
    print("✅ Сбор данных завершён.")


def run_scheduler() -> None:
    """
    Настраивает автоматический запуск задачи парсинга каждый день в 19:00.

    Использует библиотеку schedule для отслеживания времени
    и функцию time.sleep(), чтобы не перегружать систему.

    Returns
    -------
    None

    Examples
    --------
    >>> run_scheduler()  # будет ждать запуска в 19:00 каждый день
    """
    # ⏰ для теста можно временно поставить, например, "21:35"
    schedule.every().day.at("19:00").do(job)
    print("📅 Планировщик запущен. Ожидание времени для запуска задачи...")

    while True:
        schedule.run_pending()
        time.sleep(60)  # проверяем раз в минуту

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

]9;4;3;\============================= test session starts ==============================
platform darwin -- Python 3.13.7, pytest-9.0.0, pluggy-1.6.0
rootdir: /Users/ekaterina/Documents/МФТИ/Программирование/projects/books_scraper/notebooks
collected 0 items                                                              
]9;4;0;\
============================ no tests ran in 0.00s =============================
ERROR: file or directory not found: test/test_scraper.py



## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```